# Tutorial d'initiation Neo4j

### Partie 1 : "Airport routes graph"

Tout d'abord il faut créer ta propre instance Sandbox sur https://dev.neo4j.com/sandbox. 
Sandbox Neo4j est un environnement virtuel gratuit et isolé dans lequel vous pouvez expérimenter avec la base de données Neo4j sans affecter un environnement de production. 

#### Installation des librairies necessaires :

In [ ]:
pip install neo4j

In [ ]:
pip install graphdatascience

In [70]:
import pandas as pd

from graphdatascience import GraphDataScience

#### Connexion à Neo4j :

In [73]:
url = 'bolt://3.86.234.148:7687'
pwd = 'comments-shipment-puddle'

In [74]:
gds = GraphDataScience(url, auth=("neo4j", pwd))

On peut aller sur Neo4j pour visualiser le schéma du graphe et lancer la commande Cypher suivante :                          CALL db.schema.visualization()
Maintenant qu'on est connecté sur Neo4j, nous allons lancer une requete cypher pour vérifier la taille de notre graphe.

In [75]:
query = 'MATCH (n) RETURN COUNT(n) AS count'
gds.run_cypher(query)

,count
0,8628


#### Analyse exploratoire de données :

Examiner la distribution du nombre d'aéroports par continent :

In [76]:
query = 'MATCH (:Airport)-[:ON_CONTINENT]->(c:Continent) RETURN c.name AS continentName, count(*) AS numAirports ORDER BY numAirports DESC'
gds.run_cypher(query)

,continentName,numAirports
0,NA,989
1,AS,971
2,EU,605
3,AF,321
4,SA,313
5,OC,304


Calculer le minimum, le maximum, la moyenne et l'écart-type du nombre de vols au départ de chaque aéroport :

In [77]:
query ='MATCH (a:Airport)-[:HAS_ROUTE]->(:Airport) WITH a, count(*) AS numberOfRoutes RETURN min(numberOfRoutes), max(numberOfRoutes), avg(numberOfRoutes), stdev(numberOfRoutes)'
gds.run_cypher(query)

,min(numberOfRoutes),max(numberOfRoutes),avg(numberOfRoutes),stdev(numberOfRoutes)
0,1,307,20.905363,38.287309


Nous pouvons obtenir les statistiques sur la longueur des vols entre les aéroports :

In [78]:
query ='MATCH (:Airport)-[r:HAS_ROUTE]->(:Airport) WITH r.distance AS routeDistance RETURN min(routeDistance), max(routeDistance), avg(routeDistance), stdev(routeDistance)'
gds.run_cypher(query)

,min(routeDistance),max(routeDistance),avg(routeDistance),stdev(routeDistance)
0,2,9523,1262.169307,1336.52411


#### Projection de graphe :

La première étape pour exécuter n'importe quel algorithme GDS dans Neo4j consiste à créer une projection de graphe sous un nom défini par l'utilisateur. 
Ces projections de graphe, enregistrées dans le catalogue de graphes sous un nom spécifique choisi par l'utilisateur, sont des sous-ensembles de notre graphe complet. 
Elles sont conçues pour être utilisées dans le calcul des résultats à l'aide des algorithmes GDS. 
Une première méthode est d'utiliser une requête cypher pour creer cette projection :

In [80]:
query = """CALL gds.graph.project(
                'routes2',
                'Airport',
                'HAS_ROUTE'
            )
            YIELD
                graphName, nodeProjection, nodeCount, relationshipProjection, relationshipCount
"""

gds.run_cypher(query)

,graphName,nodeProjection,nodeCount,relationshipProjection,relationshipCount
0,routes2,"{'Airport': {'label': 'Airport', 'properties':...",3503,"{'HAS_ROUTE': {'orientation': 'NATURAL', 'inde...",46389


Une deuxième méthode est d'utiliser le client GDS :

In [81]:
G, res = gds.graph.project(
    'airports',                 
    'Airport',   
    "HAS_ROUTE"                            
)

Nous avons crée une projection de graphe appelé "airport" sur les noeuds de type "Airport" et les relations "HAS_ROUTE". Cette requête crée deux variables : 
-"G" : Cette variable contient le graphe projeté résultant de la requête gds.graph.project. C'est une instance de graphe dans le cadre de Neo4j. On va utiliser G pour exécuter des algorithmes graphiques tels que PageRank sur ce graphe.
-"res" : Cette variable contient des informations sur la projection du graphe et éventuellement des métadonnées. Elle est utilisée pour stocker le résultat ou les détails de l'opération de projection du graphe. 

In [82]:
type(res)

pandas.core.series.Series

In [83]:
print(res)

nodeProjection            {'Airport': {'label': 'Airport', 'properties':...
relationshipProjection    {'HAS_ROUTE': {'orientation': 'NATURAL', 'inde...
graphName                                                          airports
nodeCount                                                              3503
relationshipCount                                                     46389
projectMillis                                                           124
Name: 0, dtype: object


On a crée l'objet graphe G et on peut utiliser plusieurs méthodes sur ce graphe. On peut compter le nombre de noeuds et de relations dans ce graphe (uniquement les noeuds "Airport" et les relation "HAS-ROUTE")

In [86]:
G.node_count(), G.relationship_count()

(3503, 46389)

On peut calculer la distribution des degrés des nœuds dans le graphe G :

In [89]:
G.degree_distribution()

p99     175.000000
min       0.000000
max     307.000000
mean     13.242649
p90      38.000000
p50       2.000000
p999    282.000000
p95      71.000000
p75       9.000000
dtype: float64

On peut aussi appliquer les algorithmes de graphe qui existents sur Neo4j. Ici on va appliquer le PageRank qui est utilisé pour évaluer l'importance relative des nœuds dans un graphe. Le calcul du PageRank dans Neo4j repose sur le principe que les nœuds qui sont liés à de nombreux autres nœuds importants sont eux-mêmes importants. L'algorithme attribue à chaque nœud un score de PageRank en fonction de ces liens et de la manière dont ils sont connectés à d'autres nœuds. 

Le mode "stats" peut être utile pour évaluer les performances d'un algorithme sans modifier le graphe en mémoire. Lorsque vous exécutez un algorithme dans ce mode, une seule ligne contenant un résumé des statistiques de l'algorithme est renvoyée.

In [98]:
result = gds.pageRank.stats(
    G,
    maxIterations=20,
    dampingFactor=0.85
)

print(result)

ranIterations                                                            20
didConverge                                                           False
centralityDistribution    {'p99': 5.497771263122559, 'min': 0.1499996185...
postProcessingMillis                                                     67
preProcessingMillis                                                       0
computeMillis                                                           240
configuration             {'jobId': '456e91b5-443c-4e2f-b038-f05183458ac...
Name: 0, dtype: object


Le mode "stream" renvoie les résultats d'un algorithme sous forme de lignes de résultats Cypher. Cela est similaire à la manière dont fonctionnent les requêtes de lecture Cypher standard.

In [99]:
results = gds.pageRank.stream(
    G,
    maxIterations=20,
    dampingFactor=0.85
)

print(results)

      nodeId      score
0       8627  10.389948
1       8628   2.651157
2       8629   3.397036
3       8630   2.913093
4       8631   5.587850
...      ...        ...
3498   12125   0.174984
3499   12126   0.174984
3500   12127   0.174984
3501   12128   0.174984
3502   12129   4.808279

[3503 rows x 2 columns]


Le mode "mutate" agit sur le graphe en mémoire et le met à jour avec une nouvelle propriété spécifiée à l'aide du paramètre de configuration "mutateProperty". La nouvelle propriété ne doit pas déjà exister dans le graphe en mémoire.

In [100]:
result = gds.pageRank.mutate(
    G,
    mutateProperty="pageRankScore",
    maxIterations=20,
    dampingFactor=0.85
)

print(result)

mutateMillis                                                              2
nodePropertiesWritten                                                  3503
ranIterations                                                            20
didConverge                                                           False
centralityDistribution    {'p99': 5.497771263122559, 'min': 0.1499996185...
postProcessingMillis                                                    192
preProcessingMillis                                                       0
computeMillis                                                           307
configuration             {'jobId': 'c6b3255f-e68b-40c9-af29-8789658ad09...
Name: 0, dtype: object


Le mode "écriture" écrit les résultats du calcul de l'algorithme dans la base de données Neo4j. Les données écrites peuvent être des propriétés de nœuds (comme les scores PageRank), de nouvelles relations (comme les similarités entre nœuds dans le cas de la Similarité entre Nœuds), ou des propriétés de relations (uniquement pour les relations nouvellement créées).

In [101]:
result = gds.pageRank.write(
    G,
    writeProperty="pageRankScore",
    maxIterations=20,
    dampingFactor=0.85
)

print(result)

writeMillis                                                            1472
nodePropertiesWritten                                                  3503
ranIterations                                                            20
didConverge                                                           False
centralityDistribution    {'p99': 5.497771263122559, 'min': 0.1499996185...
postProcessingMillis                                                    187
preProcessingMillis                                                       0
computeMillis                                                           213
configuration             {'maxIterations': 20, 'writeConcurrency': 4, '...
Name: 0, dtype: object


Après avoir suivi l'exemple, à la fois les graphes en mémoire et les données dans la base de données Neo4j peuvent être supprimés.

In [102]:
result = gds.graph.drop(G)
print(result)

gds.run_cypher("""
    MATCH (n)
    DETACH DELETE n
""")

graphName                                                         airports
database                                                             neo4j
memoryUsage                                                               
sizeInBytes                                                             -1
nodeCount                                                             3503
relationshipCount                                                    46389
configuration            {'relationshipProjection': {'HAS_ROUTE': {'ori...
density                                                           0.003781
creationTime                           2023-09-07T08:08:06.562479904+00:00
modificationTime                       2023-09-07T09:07:40.679523334+00:00
schema                   {'graphProperties': {}, 'relationships': {'HAS...
schemaWithOrientation    {'graphProperties': {}, 'relationships': {'HAS...
Name: 0, dtype: object


""


On ferme la connexion :

In [103]:
gds.close()

### Partie 2 : Importer un des données RDF à partir de Wikidata 

#### Connection à une Blank Sanbox : 

In [50]:
url = 'bolt://44.205.7.175:7687'
pwd = 'hull-offenders-trips'

In [51]:
gds = GraphDataScience(url, auth=("neo4j", pwd))

#### Intialiser Neosemantics :

In [52]:
query = 'CALL n10s.graphconfig.init({handleVocabUris: "IGNORE"})'
gds.run_cypher(query)

,param,value
0,handleVocabUris,IGNORE
1,handleMultival,OVERWRITE
2,handleRDFTypes,LABELS
3,keepLangTag,False
4,keepCustomDataTypes,False
5,applyNeo4jNaming,False
6,baseSchemaNamespace,neo4j://graph.schema#
7,baseSchemaPrefix,n4sch
8,classLabel,Class
9,subClassOfRel,SCO


#### Creation de la contrainte d'unicité des URI :

In [53]:
query = '''
CREATE CONSTRAINT n10s_unique_uri FOR (r:Resource) REQUIRE r.uri IS UNIQUE
  '''
gds.run_cypher(query)

""


#### Inspection préliminaire du graphe :

Nous allons importer des données RDF à partir de Wikidata. Nous allons utiliser l'API Wikidata Query Service (https://query.wikidata.org). Nous allons utiliser une requête qui interroge la base de données Wikidata pour obtenir les noms des restaurants situés en France, en spécifiant également la ville où ils se trouvent. La requête SPARQL est la suivante : 

In [58]:
query = '''
WITH 'PREFIX sch: <http://schema.org/> 
CONSTRUCT{ ?item a sch:Restaurant; 
            sch:name ?restaurantName; 
            sch:location ?location.            	
            ?location a sch:City; 
           	sch:name ?cityName  } 
WHERE { ?item wdt:P31/wdt:P279* wd:Q11707.
        ?item wdt:P17 wd:Q142 .
        ?item rdfs:label ?restaurantName .
        ?item wdt:P131 ?location .
        ?location rdfs:label ?cityName 
        filter(lang(?cityName)="en")   }
' AS sparql
CALL n10s.rdf.preview.fetch(
  "https://query.wikidata.org/sparql?query=" +
     apoc.text.urlencode(sparql),"JSON-LD",
  { headerParams: { Accept: "application/ld+json"}} )
YIELD nodes, relationships
RETURN nodes, relationships
'''
gds.run_cypher(query)

,nodes,relationships
0,"[(name, uri), (name, uri), (name, uri), (name,...","[(), (), (), (), (), (), (), (), (), (), (), (..."


#### Insertion des données dans le graphe : 

In [59]:
query = '''
WITH 'PREFIX sch: <http://schema.org/> 
CONSTRUCT{ ?item a sch:Restaurant; 
            sch:name ?restaurantName; 
            sch:location ?location.            	
            ?location a sch:City; 
           	sch:name ?cityName  } 
WHERE { ?item wdt:P31/wdt:P279* wd:Q11707.
        ?item wdt:P17 wd:Q142 .
        ?item rdfs:label ?restaurantName .
        ?item wdt:P131 ?location .
        ?location rdfs:label ?cityName 
        filter(lang(?cityName)="en")   }
        ' AS sparql
CALL n10s.rdf.import.fetch(
  "https://query.wikidata.org/sparql?query=" +
     apoc.text.urlencode(sparql),"JSON-LD",
  { headerParams: { Accept: "application/ld+json"}} )
YIELD terminationStatus, triplesLoaded
RETURN terminationStatus, triplesLoaded
'''
gds.run_cypher(query)


,terminationStatus,triplesLoaded
0,OK,9605


#### Vérifier que les noeuds existent :

In [60]:
query = '''
MATCH (n:Restaurant) 
RETURN COUNT(*)
'''
gds.run_cypher(query)

,COUNT(*)
0,525
